In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [2]:
import nest_asyncio

from llama_parse import LlamaParse  # pip install llama-parse
import os
llamaindex_key = os.getenv('llamaindex')

nest_asyncio.apply()

parser = LlamaParse(
    api_key=llamaindex_key, 
    result_type="markdown"  # "markdown" and "text" are available
)

documents = parser.load_data("E:/Code/chat/gdpr/pdf/guidelines/wp251rev_01_en_A754F3E1-FB46-9E76-C0A919864E4B6641_49826.pdf") 

file = "../tmp/decision_making_llama.md"
with open(file, 'w', encoding='utf-8') as f:
    f.write(documents[0].text)

Started parsing the file under job_id 11b62271-8ec0-4088-88ee-1138b2f81c01
.........

In [2]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'wp251rev_01_en_A754F3E1-FB46-9E76-C0A919864E4B6641_49826.pdf'
paper_name = 'Guidelines on Automated individual decision-making and Profiling for the purposes of Regulation 2016/679'
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

In [9]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 4, end_page = 0, header_size=70, footer_size=90, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

file = "../tmp/decision_making.txt"
with open(file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../../original/decision_making.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)

In [3]:
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""
section_pattern = re.compile(r'\b(I|II|III|IV|V|VI)\.\s*(.*)')
subsection_pattern = re.compile(r'([A-Z])\.\s*(.*)') 
point_pattern = re.compile(r'(\d+)\.\s*(.*)$') 
annex_pattern = re.compile(r'^ANNEX (\d+) - (.+)$') # ANNEX, number,  - ,  text

table_pattern = re.compile(r'\|')
# table_entry_number_pattern = re.compile(r'(\d+(\.\d+)?(\.\d+)?)\s+(.+)')
# table_section = re.compile(r'<td colspan=6>(\d+) - (.+)$')
# table_subsection = re.compile(r'<td colspan=6>\s*(.+)\s*')


data = []
table_data = []
for entry in doc_as_array:
    section_match = section_pattern.match(entry)
    subsection_match = subsection_pattern.match(entry)
    point_match = point_pattern.match(entry)
    annex_match = annex_pattern.match(entry)
    table_match = table_pattern.match(entry)
    if section_match:
        match = section_match
        section = match.group(1)
        subsection = ""
        point = ""
        heading = True
        text = match.group(2)
        section_reference = section
        data.append([section, subsection, point, heading, text, section_reference])
    elif subsection_match:
        match = subsection_match
        section = section
        subsection = match.group(1)
        point = ""
        heading = True
        text = match.group(2)
        section_reference = section + "." + subsection
        data.append([section, subsection, point, heading, text, section_reference])
    elif point_match:
        match = point_match
        section = section
        subsection = subsection
        point = match.group(1)
        heading = True
        text = match.group(2)
        section_reference = section + "." + subsection + "." + point
        data.append([section, subsection, point, heading, text, section_reference])
    elif annex_match:
        match = annex_match
        section = "Annex " + match.group(1)
        subsection = ""
        point = ""
        heading = True
        text = match.group(2)
        section_reference = section
        data.append([section, subsection, point, heading, text, section_reference])
    # elif table_match:
    #     pass
        # components = [component.strip() for component in entry.strip().strip('|').split('|')]
        # if len(components) == 5:
        #     table_entry_number_match = table_entry_number_pattern.match(components[0])
        #     if table_entry_number_match:
        #         components = [table_entry_number_match.group(1), table_entry_number_match.group(4)] + components[1:]
        #         table_data.append(components)
        #     else:
        #         print(f"Row removed from table_data: {components}")
        #         #components = [""] + components
        # elif len(components) == 1:
        #     table_section_match = table_section.match(components[0])
        #     table_subsection_match = table_subsection.match(components[0])
        #     if table_section_match:
        #         components = [table_section_match.group(1), table_section_match.group(2)]
        #         table_data.append(components)
        #     elif table_subsection_match:
        #         print(f"Row removed from table_data: {components}")
        #         # components = ["", table_subsection_match.group(1)]
        #     else:
        #         raise AttributeError(f"A row with one element did not match the required patterns: {entry}")
        # else:
        #     raise AttributeError("A table entry did not have the correct number of elements")
    else:
        section = section
        subsection = subsection
        point = point
        heading = False
        text = entry
        section_reference = section
        if subsection:
            if point:
                section_reference = section + "." + subsection + "." + point
            else:
                section_reference = section + "." + subsection
        
        data.append(["", "", "", heading, text, section_reference])



df = pd.DataFrame(data, columns = columns)
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [5]:
#df_no_table = df[~df["section_reference"].str.contains("Annex")]
df_no_table = df

In [6]:
print(df_no_table.iloc[-50])

section                                          
subsection                                       
point                                            
heading                                     False
text                 |Article| Recital| Comments|
section_reference                         Annex 2
Name: 351, dtype: object


In [7]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df_no_table.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df_no_table.at[index, "text"] = augmented_note
        #print

Row 1 augmented with footnotes
Row 40 augmented with footnotes
Row 71 augmented with footnotes
Row 75 augmented with footnotes
Row 77 augmented with footnotes
Row 79 augmented with footnotes
Row 87 augmented with footnotes
Row 91 augmented with footnotes
Row 92 augmented with footnotes
Row 113 augmented with footnotes
Row 117 augmented with footnotes
Row 119 augmented with footnotes
Row 125 augmented with footnotes
Row 130 augmented with footnotes
Row 133 augmented with footnotes
Row 136 augmented with footnotes
Row 137 augmented with footnotes
Row 143 augmented with footnotes
Row 150 augmented with footnotes
Row 156 augmented with footnotes
Row 159 augmented with footnotes
Row 161 augmented with footnotes
Row 162 augmented with footnotes
Row 166 augmented with footnotes
Row 172 augmented with footnotes
Row 180 augmented with footnotes
Row 181 augmented with footnotes
Row 189 augmented with footnotes
Row 197 augmented with footnotes
Row 200 augmented with footnotes
Row 203 augmented wi

In [8]:
print(df.iloc[-50]["text"])

|Article| Recital| Comments|


In [9]:
file = "../../inputs/documents/decision_making.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [13]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.decision_making
importlib.reload(gdpr_rag.documents.decision_making)
from gdpr_rag.documents.decision_making import DecisionMaking

path_to_manual_as_csv_file = "../../inputs/documents/decision_making.parquet"

doc = DecisionMaking(path_to_manual_as_csv_file)


In [16]:
from IPython.display import Markdown, display

section = "II"
section = "I"
section = "II.B"
section = "II.C"
section = "III.B.6"
#section = "Annex 2"

# section = "IV.E.1"
# section = "III.A.1"
# section = "III.D.4"

# section = "IV"
# section = "IV.E"
# section = "IV.E.1"

print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


III General provisions on profiling and automated decision-making
III.B Lawful bases for processing
III.B.6 Article 6(1) (f) - necessary for the legitimate interests[^18] pursued by the controller or by a third party


# III General provisions on profiling and automated decision-making

## III.B Lawful bases for processing

### III.B.6 Article 6(1) (f) - necessary for the legitimate interests[^18] pursued by the controller or by a third party

Profiling is allowed if it is necessary for the purposes of the legitimate interests[^19] pursued by the controller or by a third party. However, Article 6(1) (f) does not automatically apply just because the controller or third party has a legitimate interest. The controller must carry out a balancing exercise to assess whether their interests are overridden by the data subject's interests or fundamental rights and freedoms.

The following are particularly relevant:

- the level of detail of the profile (a data subject profiled within a broadly described cohort such as 'people with an interest in English literature', or segmented and targeted on a granular level);

- the comprehensiveness of the profile (whether the profile only describes a small aspect of the data subject, or paints a more comprehensive picture);

- the impact of the profiling (the effects on the data subject); and

- the safeguards aimed at ensuring fairness, non-discrimination and accuracy in the profiling process.

Although the WP29 opinion on legitimate interests[^20] is based on Article 7 of the data protection Directive 95/46/EC (the Directive), it contains examples that are still useful and relevant for controllers carrying out profiling. It also suggests it would be difficult for controllers to justify using legitimate interests as a lawful basis for intrusive profiling and tracking practices for marketing or advertising purposes, for example those that involve tracking individuals across multiple websites, locations, devices, services or  data-brokering.

The controller should also consider the future use or combination of profiles when assessing the validity of processing under Article 6(1) (f).

  
[^18]: Legitimate interests listed in GDPR Recital 47 include processing for direct marketing purposes and processing strictly necessary for the purposes of preventing fraud.  
[^19]: The controller's "legitimate interest" cannot render profiling lawful if the processing falls within the Article 22(1) definition.  
[^20]: Article 29 Data Protection Working Party. Opinion 06/2014 on the notion of legitimate interests of the data controller under Article 7 of Directive 95/46/EC. European Commission, 9 April 2014, Page 47, examples on pages 59 and 60 http://ec.europa.eu/justice/data-protection/article-29/documentation/opinion- recommendation/files/2014/wp217_en.pdf . Accessed 24 April 2017